In [1]:
import numpy as np
import pandas as pd
import requests
import json
import io
import humanize as hm
import pprint
pp = pprint.PrettyPrinter(depth=6)

In [2]:
PKL_PTH = 'RO_SEA_SpeadTesting_RawData_Merged.pkl'

GIST_PREFIX = 'https://gist.github.com/xdanger/2e2f10c33c9d5ff29949b7394efce1eb/raw/bfe72866ad54faa6dfbc42c91ab3a4c6972b48a1/'
LINE_LIST = {
  'ali': [GIST_PREFIX + 'aliyun-analytics.20200911.log', '#f4b400'],
  'aws': [GIST_PREFIX + 'aws-analytics.20200911.log', '#4185f4'],
  'gcp': [GIST_PREFIX + 'gcp-analytics.20200911.log', '#db4437'],
  'upx': [GIST_PREFIX + 'pathx-analytics.20200911.log', '#0f9d58']
}
frames = []
for k, v in LINE_LIST.items():
  print("Processing: %s" % k)
  text = requests.get(v[0]).text
  data = []
  for line in io.StringIO(text):
    try:
      data.append(json.loads(line))
    except Exception:
      print(line.strip())
  rs = pd.DataFrame(data)
  rs['network'] = k
  #rs['color'] = v[1]
  frames.append(rs)
pd.concat(frames).to_pickle('RO_SEA_SpeadTesting_RawData_Merged.pkl')

Processing: ali
Processing: aws
Processing: gcp
{"time":1599808340,"srcip":"203.116.141.213",-1,"country":"Singapore","city":"Queenstown Estate"}
{"time":1599808420,"srcip":"203.116.141.213",-1,"country":"Unknown","city":"Unknown"}
Processing: upx


In [3]:
df = pd.read_pickle(PKL_PTH).sample(frac=1)
df = df[['srcip', 'country', 'network', 'speed_upload', 'speed_download']]
df.columns = ['srcip', 'country', 'network', 'speed_up', 'speed_dl']

In [4]:
import os
import geoip2.database
import maxminddb
import urllib

GEOLITE2_MMDB = 'GeoLite2-City.mmdb'
if not os.path.exists(GEOLITE2_MMDB):
  urllib.request.urlretrieve('https://macaulish.net/utils/GeoLite2-City.mmdb', GEOLITE2_MMDB)

def lookup_country(rd, srcip):
  resp = rd.get(srcip)
  if type(resp) == dict:
    try:
      return resp['country']['names']['en']
    except KeyError as inst:
      print("IP [%s]: %s" % (srcip, resp))
      return 'Unkown'
  else:
    print("IP [%s] is Unknown: %s" % (srcip, resp))
    return 'Unknown'

with maxminddb.open_database(GEOLITE2_MMDB) as reader:
  df['country'] = df['srcip'].apply(lambda x: lookup_country(reader, x))

df.to_pickle('RO_SEA_SpreadTesting_ToAnalytics.pkl')

IP [103.146.75.227] is Unknown: None
IP [193.53.99.82]: {'continent': {'code': 'EU', 'geoname_id': 6255148, 'names': {'de': 'Europa', 'en': 'Europe', 'es': 'Europa', 'fr': 'Europe', 'ja': 'ヨーロッパ', 'pt-BR': 'Europa', 'ru': 'Европа', 'zh-CN': '欧洲'}}, 'location': {'accuracy_radius': 50, 'latitude': 47.0, 'longitude': 8.0, 'time_zone': 'Europe/Vaduz'}}
IP [193.53.99.82]: {'continent': {'code': 'EU', 'geoname_id': 6255148, 'names': {'de': 'Europa', 'en': 'Europe', 'es': 'Europa', 'fr': 'Europe', 'ja': 'ヨーロッパ', 'pt-BR': 'Europa', 'ru': 'Европа', 'zh-CN': '欧洲'}}, 'location': {'accuracy_radius': 50, 'latitude': 47.0, 'longitude': 8.0, 'time_zone': 'Europe/Vaduz'}}
IP [103.146.75.179] is Unknown: None
IP [103.144.156.159] is Unknown: None
IP [103.146.66.36] is Unknown: None
IP [103.145.159.246] is Unknown: None
IP [103.150.10.197] is Unknown: None
IP [103.146.66.36] is Unknown: None
IP [193.53.99.82]: {'continent': {'code': 'EU', 'geoname_id': 6255148, 'names': {'de': 'Europa', 'en': 'Europe', 

In [8]:
Countries = df.set_index(['country', 'network']).count(level='country').reset_index(
          ).sort_values(by=['speed_dl'], ascending=False).head(6)['country'].to_numpy()

In [6]:
mean_all = df.groupby(['country', 'network']).median()
df_mean = mean_all.loc[Countries]
df_mean = pd.DataFrame(df_mean.to_records())
df_mean = df_mean.melt(id_vars=['country', 'network'])
df_mean.columns = ['country', 'network', '↕', 'speed']

In [7]:
import altair as alt
from altair import expr, datum
from altair_saver import save

sl = alt.Scale(domain=(0, 10**8/2), clamp=True)
af = alt.Axis(format='~s')
alt.data_transformers.disable_max_rows()

charts = []
for c in Countries:
  dat = df[df['country'] == c].head(7000)
  cht = alt.Chart(dat).transform_calculate(
    D='datum.speed_dl',
    U='datum.speed_up',
    T='(datum.speed_dl + datum.speed_up)'
  ).mark_point().encode(
    alt.X('D:Q', title='↓ Speed', axis=af, scale=sl),
    alt.Y('U:Q', title='→ Speed', axis=af, scale=sl),
    color=alt.Color('network:N', title='玩家线路'),
    #shape=alt.Shape('network:N', title='玩家线路'),
    opacity=alt.Opacity('T:Q', title='上传+下载速度'),
    size=alt.Size('T:Q', title='上传+下载速度')
  ).properties(
    title=c,
    width=640,
    height=640
  )
  cht.configure_header(
    titleColor='green',
    titleFontSize=14,
    labelColor='red',
    labelFontSize=14
  )
  bar = alt.Chart(df_mean[df_mean['country'] == c]).mark_bar().encode(
    alt.X('speed:Q', axis=af, scale=sl),
    y='↕:N',
    color='network:N',
    row='network:N'
  ).properties(
    width=640,
    height=30
  )
  charts.append((cht + cht.transform_loess('D', 'U', groupby=['network']).mark_line()) & bar)

'''
def to_matrix(l, n):
  return [l[i:i+n] for i in range(0, len(l), n)]
charts = to_matrix(charts, 3)
alt.concat(*charts)
'''

alt.vconcat(*charts)

alt.VConcatChart(...)